In [1]:
# Run this cell first

!git clone https://github.com/Mlodko/reed-solomon.git
%cd reed-solomon

Cloning into 'reed-solomon'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 11), reused 26 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 14.81 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/reed-solomon


In [2]:
# Install and test rich output
%pip install rich
!python -m rich

                                 Rich features                                  
                                                                                
    Colors    ✓ 4-bit color                 ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ 
              ✓ 8-bit color                 ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ 
              ✓ Truecolor (16.7 million)    ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ 
              ✓ Dumb terminals              ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ 
              ✓ Automatic color conversion  ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ 
                                                                                
    Styles    All ansi styles: bold, dim, italic, underline, strikethrough,     
              reverse, and even blink.                                          
                                                                                
     Text     Word wrap text. Justify left, center, right or full.              
                            

In [4]:
from rs import RSCoder, binary_format
from rich.console import Console
from rich.text import Text
import math
import random
# =======================
# 🎨 Color Parameters
# =======================
COLOR_BG = "plum2"
COLOR_PAD = "bright_black"
COLOR_MSG = "green"
COLOR_PARITY = "steel_blue"
STYLE_COMMON = f"bold on {COLOR_BG}"  # shared background with bold
COLOR_CHECK_OK = "bright_green bold"
COLOR_CHECK_FAIL = "red bold"

# =======================
# 📥 Parameters for Colab
# =======================
n = 255  # @param {type:"slider", min:1, max:255, step:1}
k = 223  # @param {type:"slider", min:1, max:254, step:1}
input = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."  # @param {type:"string", placeholder:"Enter a message to be encoded"}
errors_per_block = 2 # @param {type:"slider", min:0, max:255, step:1}


def introduce_errors(encoded_blocks, errors_per_block=2):
    corrupted_blocks = []
    for block in encoded_blocks:
        block_bytes = bytearray(block.encode('latin1'))  # Use latin1 to preserve byte values
        error_positions = random.sample(range(len(block_bytes)), errors_per_block)
        for pos in error_positions:
            block_bytes[pos] = (block_bytes[pos] + random.randint(1, 255)) % 256
        corrupted_blocks.append(block_bytes.decode('latin1'))
    return corrupted_blocks

# =======================
# 🚧 Encode & Decode
# =======================
coder = RSCoder(n, k)

input_char_length = len(input)
input_blocks = [input[i:i + k] for i in range(0, input_char_length, k)]
encoded_blocks = [coder.encode(block) for block in input_blocks]
encoded_blocks = introduce_errors(encoded_blocks, errors_per_block)

# Decode each block individually, then strip padding
decoded_blocks = []
for i, encoded_block in enumerate(encoded_blocks):
    decoded_block = coder.decode(encoded_block)
    # Remove padding: only keep actual message characters from the original block
    decoded_blocks.append(decoded_block[:len(input_blocks[i])])

decoded = ''.join(decoded_blocks)

# =======================
# 🔢 Binary representations
# =======================
bin_input = binary_format(input)
bin_decoded = binary_format(decoded)
bin_input_blocks = [binary_format(block) for block in input_blocks]
bin_encoded_blocks = [binary_format(block) for block in encoded_blocks]

# =======================
# 📦 Block Highlighting
# =======================
console = Console(highlight=False)
highlighted_blocks = []

for block, encoded_block in zip(input_blocks, encoded_blocks):
    pad_len = k - len(block)
    byte_lines = binary_format(encoded_block).strip().splitlines()
    byte_tokens = [b for line in byte_lines for b in line.strip().split()]

    text = Text()

    for i, byte in enumerate(byte_tokens):
        if i < pad_len:
            style = f"{COLOR_PAD} {STYLE_COMMON}"
        elif i < k:
            style = f"{COLOR_MSG} {STYLE_COMMON}"
        else:
            style = f"{COLOR_PARITY} {STYLE_COMMON}"

        text.append(byte, style)
        text.append(" " if (i + 1) % 16 else "\n", f"on {COLOR_BG}")

    highlighted_blocks.append(text)

# =======================
# 🎨 Styled Inputs
# =======================
bin_input_pretty = Text(bin_input)
bin_input_pretty.stylize(f"{COLOR_MSG} bold", 0, len(bin_input))

bin_decoded_pretty = Text(bin_decoded)
start = bin_decoded.find(bin_input)
if start != -1:
    bin_decoded_pretty.stylize(f"{COLOR_MSG} bold", start, start + len(bin_input))

# =======================
# 📊 Stats
# =======================
block_count = math.ceil(input_char_length / k)
encoded_char_length = block_count * n
padding_char_length = block_count * k - input_char_length
parity_char_length = block_count * (n - k)

# =======================
# 📤 Output
# =======================
console.print("Unencoded message:\n\tBinary representation:")
console.print(bin_input_pretty)
console.print(f"Character (byte) length: {input_char_length}")

console.print("\n=========================================\n")
console.print("Encoded message:\n\tBinary representation:")
for block in highlighted_blocks:
    console.print(block + "\n")

console.print(f"Total byte length (block_count × n): [{COLOR_BG}]{encoded_char_length}[/{COLOR_BG}]")
console.print(f"Padding byte length (block_count × k - input_chars): [{COLOR_PAD}]{padding_char_length}[/{COLOR_PAD}]")
console.print(f"Message byte length: [{COLOR_MSG}]{input_char_length}[/{COLOR_MSG}]")
console.print(f"Parity byte length (block_count × (n - k)): [{COLOR_PARITY}]{parity_char_length}[/{COLOR_PARITY}]")

console.print("\n=========================================\n")
console.print("Decoded message:\n\tBinary representation:")
console.print(bin_decoded_pretty)

console.print("\n=========================================\n")
console.print("Decoding validity check:")
if decoded == input:
  console.print(f"[{COLOR_CHECK_OK}]\[OK]Decoded message matches input[/{COLOR_CHECK_OK}]")
else:
    console.print(f"[{COLOR_CHECK_FAIL}]\[FAIL]Decoded message matches input[/{COLOR_CHECK_FAIL}]")
    console.print(f"\tInput message: {input}\n\tDecoded message: {decoded}")



RecursionError: maximum recursion depth exceeded